### Q1. Install MLflow
```
To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?
````

In [1]:
import mlflow

mlflow.__version__

'2.21.3'

### Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7


# Answer is 4

## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework). 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* 2
* 4
* 8
* 10

Default value is 2

can check by running below in cwd
```
python -c "from sklearn.ensemble import RandomForestRegressor; print(RandomForestRegressor().get_params()['min_samples_split'])"
```


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`



### Run below

mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./artifacts \
    --serve-artifacts
### Answer is default-artifact-root

## Q5. Tune model hyperparameters


Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818
* 6.336

In [3]:
# The best loss is 5.335

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

mlflow.set_tracking_uri("http://127.0.0.1:5000")
client = MlflowClient()



experiment_name = "random-forest-hyperopt"
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id


runs = client.search_runs(
    experiment_ids=[experiment_id],
    run_view_type=ViewType.ACTIVE_ONLY,
    order_by=["metrics.rmse ASC"],  # Smallest RMSE first
    max_results=1
)


best_run = runs[0]
print(f"Best run ID: {best_run.info.run_id}")
print(f"Best RMSE: {best_run.data.metrics['rmse']:.4f}")
print("Hyperparameters:")
for key, value in best_run.data.params.items():
    print(f"  {key}: {value}")




Best run ID: 24b68d2c01d14325a2029b02f62493cf
Best RMSE: 5.3354
Hyperparameters:
  max_depth: 19
  min_samples_leaf: 2
  min_samples_split: 2
  n_estimators: 11
  random_state: 42


In [4]:
import pandas as pd

df = mlflow.search_runs(experiment_ids=[experiment_id])
df[['run_id', 'metrics.rmse'] + [c for c in df.columns if c.startswith("params.")]].sort_values("metrics.rmse").head()


,run_id,metrics.rmse,params.n_estimators,params.max_depth,params.min_samples_split,params.min_samples_leaf,params.random_state
4,24b68d2c01d14325a2029b02f62493cf,5.335420,11,19,2,2,42
9,b8fb164f4aba4fd99bdd1d59848a4886,5.354695,40,15,3,2,42
0,1f422d7502354eb9b377bc5e049318d5,5.355042,19,20,9,1,42
11,e99276ded430495fa8030ecba193b3a6,5.357491,26,14,4,3,42
5,4890c26ec87b4ac39f060cd39a2924df,5.363360,23,14,6,2,42


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568


# The best loss is 5.335

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

mlflow.set_tracking_uri("http://127.0.0.1:5000")
client = MlflowClient()



experiment_name = "random-forest-best-models"
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id


runs = client.search_runs(
    experiment_ids=[experiment_id],
    run_view_type=ViewType.ACTIVE_ONLY,
    order_by=["metrics.rmse ASC"],  # Smallest RMSE first
    max_results=1
)


best_run = runs[0]
print(f"Best run ID: {best_run.info.run_id}")
print(f"Best RMSE: {best_run.data.metrics['rmse']:.4f}")
print("Hyperparameters:")
for key, value in best_run.data.params.items():
    print(f"  {key}: {value}")




In [6]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

mlflow.set_tracking_uri("http://127.0.0.1:5000")
client = MlflowClient()

experiment_name = "random-forest-best-models"
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

# Sort by test RMSE, not "rmse"
runs = client.search_runs(
    experiment_ids=[experiment_id],
    run_view_type=ViewType.ACTIVE_ONLY,
    order_by=["metrics.test_rmse ASC"],
    max_results=1
)

best_run = runs[0]
print(f"✅ Best run ID: {best_run.info.run_id}")
print(f"🏆 Best Test RMSE: {best_run.data.metrics['test_rmse']:.4f}")
print("📊 Hyperparameters:")
for key, value in best_run.data.params.items():
    print(f"  {key}: {value}")


✅ Best run ID: b7a06e9cb4a4441fbc83fc3d07dbf80d
🏆 Best Test RMSE: 5.5674
📊 Hyperparameters:
  max_depth: 19
  n_estimators: 11
  min_samples_split: 2
  min_samples_leaf: 2
  random_state: 42
